# Preperation

## Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
labels = pd.read_csv('data/traininglabels.csv')
data = pd.read_csv('data/trainingdata.csv')

wellsdf = pd.merge(labels, data, on='id')

## Clean Data
Per the rational in EDA notebook

In [3]:
wells2 = wellsdf.copy()

wells2 = wells2.drop(['funder', 'installer', 'scheme_name', 'public_meeting', 'scheme_management',
                      'id', 'amount_tsh', 'num_private', 'permit', 'subvillage'], axis=1)

# Convert numeric that should be categorical
wells2[['region_code', 'district_code']] = wells2[['region_code', 'district_code']].astype('str')

#### Simplify and convert date_recorded to year_recorded

In [4]:
wells2.date_recorded = pd.to_datetime(wells2.date_recorded)
wells2['year_recorded'] = pd.DatetimeIndex(wells2['date_recorded']).year
wells2.drop('date_recorded', axis=1, inplace=True)

##### Construction year
Remove rows with zero, average the remainder, set zeros in wells2 to that value

In [5]:
con_year = wells2.drop(wells2[wells2.construction_year == 0].index) 
con_avg = con_year.construction_year.mean().round(0)

wells2.construction_year.replace(0, con_avg, inplace=True)

#### Lat & Long
The default value for longitude (0) is outside of Tanzania (40°29' E to 29°10' E), so it will be replaced by the mean longitude from the dataset.  While the default value for latitude is possible, it may skew our data more north (-2), so it will be replaced with the mean latitude.

In [6]:
longitude = wells2.drop(wells2[wells2.longitude == 0].index) 
long_avg = longitude.longitude.mean()
wells2.longitude.replace(0, long_avg, inplace=True)

latitude = wells2.drop(wells2[wells2.latitude == -2.000000e-08].index) 
lat_avg = latitude.latitude.mean()
wells2.latitude.replace(-2.000000e-08, lat_avg, inplace=True)

# Modeling

## Create Pipelines